<a href="https://colab.research.google.com/github/hartmann-pereira/D/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configure stuff

This cell is setting up Pyspark in google-colab
1. Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java.
2. Next, we will download and unzip Apache Spark with Hadoop 2.7 to install it. [Spark version 3.2.0 (latest stable) and hadoop 2.7]
3. After this we need to import os and set the ‘environment’ path.
4. Then we need to install and import the ‘findspark’ library that will locate Spark on the system and import it as a regular library.

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 198 kB 72.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=97580e17c4e2a08156744b24e3803751fd8ebad207c477046883ce9a41d6d5e8
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Lastly import finspark as a regular library and initialize it

In [2]:
import findspark
findspark.init()

Now the final step needed is to start a Spark session, importing SparkSession from library and we want to run it localy, name this Colab for simplicity, get or create, its gonna create because this is the first time we running this.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Thats all done, let's get started with Pyspark and run a little simple project for this session

I can download the file directly into Colab using the ‘wget’ command like this to get my file from github 

# Import my dataset into my workbook and into a spark df

In [5]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2021-12-08 16:57:33--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2021-12-08 16:57:33 (19.9 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [6]:
df = spark.read.json("/tmp/sample_books.json")

In [7]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [10]:
#Let's check what our dataset looks like 
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



# Data analysis

In [12]:
#How big is this dataset?
df.count()

13

In [13]:
#Let's see a few columns of interest 
df.select("title","price","year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [15]:
#Now we could filter this dataset a bit to get some insight into some questions
#Let's get the books that are written after 1995 and cost 10 bucks
df_filtered = df.filter("year_written > 1995 AND price > 10 AND title IS NOT NULL")
df_filtered.show(59,False)

+--------------------+--------------+-----+------------+------------+
|author              |edition       |price|title       |year_written|
+--------------------+--------------+-----+------------+------------+
|Cunnningham, Michael|Harcourt Brace|12.35|The Hours   |1999        |
|Rowling, J.K.       |Harcourt Brace|19.95|Harry Potter|2000        |
+--------------------+--------------+-----+------------+------------+



In [20]:
#Doing a bit more analysis, let's get all the books from previous dataset that contain Harry Potter in the title
df_filtered.select("title").filter("title LIKE 'Harry Potter'").show()

+------------+
|       title|
+------------+
|Harry Potter|
+------------+



In [22]:
#Previous cell only provided the title, lets get all other rows
df_filtered.select("title", "edition","price", "year_written").filter("title LIKE 'Harry Potter'").show()
#if this dataset had many of the same books we could add a distinct() function at the end to only get 1 of each

+------------+--------------+-----+------------+
|       title|       edition|price|year_written|
+------------+--------------+-----+------------+
|Harry Potter|Harcourt Brace|19.95|        2000|
+------------+--------------+-----+------------+



In [23]:
#Using sparkSQL function we can get even more insight from our dataset, let's import it 
from pyspark.sql.functions import max

In [26]:
#Let's find the costliest books in our dataset
#Get the max value for the price column
maxValue = df.agg(max("price")).collect()[0][0]
print (maxValue)

29.0


In [27]:
#We have the value, but what book or books have this price (could be many)
df.select("title","price").filter(df.price == maxValue).show()

+-------------------+-----+
|              title|price|
+-------------------+-----+
|A Room of One's Own| 29.0|
+-------------------+-----+

